In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC ### 1. Movimentação: Landing Zone para Bronze
# MAGIC 
# MAGIC **Origem:** `/Volumes/movies_catalog/bronze/landing_zone_movies/top_100_movies_full_best_effort.csv`
# MAGIC 
# MAGIC **Destino:** Tabela `movies_catalog.bronze.movies_raw`
# MAGIC 
# MAGIC **Ações:**
# MAGIC 1. Lê o arquivo CSV bruto da Landing Zone.
# MAGIC 2. Adiciona colunas de metadados (`data_ingestao`, `arquivo_origem`).
# MAGIC 3. Salva os dados como uma tabela Delta na camada Bronze.

# COMMAND ----------

from pyspark.sql.functions import current_timestamp, input_file_name

# Define o catálogo e o schema a serem usados
spark.sql("USE CATALOG movies_catalog")
spark.sql("USE SCHEMA bronze")

# COMMAND ----------

# Caminho completo para o arquivo dentro do Volume
landing_path = "/Volumes/movies_catalog/bronze/landing_zone_movies/top_100_movies_full_best_effort.csv"
bronze_table_name = "movies_raw"

# Leitura do arquivo CSV bruto da Landing Zone
try:
    df_raw = spark.read.format("csv") \
                       .option("header", "true") \
                       .option("inferSchema", "true") \
                       .load(landing_path)

    # Adiciona metadados de controle para rastreabilidade
    df_bronze = df_raw.withColumn("data_ingestao", current_timestamp()) \
                      .withColumn("arquivo_origem", input_file_name())

    # Salva como uma tabela Delta gerenciada na camada Bronze
    df_bronze.write.mode("overwrite") \
             .option("overwriteSchema", "true") \
             .saveAsTable(bronze_table_name)

    print(f"Tabela '{bronze_table_name}' salva com sucesso na camada Bronze.")

except Exception as e:
    print(f"Erro ao processar o arquivo. Verifique se o caminho '{landing_path}' está correto e se o arquivo foi carregado.")
    raise e